# Assignment 7

## Problem 4.26

We will now use QR decomposition to solve a linear system of equations, $Ax = b$. This equation can be rewritten as: $QRx = b$. We can take advantage of the orthogonality of $Q$ to re-express this as: $Rx = Q^Tb$. But now the right-hand side contains only known quantities and the left-hand side has the upper-triangular matrix $R$, so a back substitution is all that’s needed. Implement this approach in Python for classical Gram–Schmidt.

In [7]:
import numpy as np

def backsub(U,bs):
  n = bs.size
  xs = np.zeros(n)
  for i in reversed(range(n)):
    xs[i] = (bs[i] - U[i,i+1:]@xs[i+1:])/U[i,i]
  return xs

def mag(xs):
 return np.sqrt(np.sum(xs*xs))

def qrdec(A):
  n = A.shape[0]
  Ap = np.copy(A)
  Q = np.zeros((n,n))
  R = np.zeros((n,n))
  for j in range(n):
    for i in range(j):
      R[i,j] = Q[:,i]@A[:,j]
      Ap[:,j] -= R[i,j]*Q[:,i]
    R[j,j] = mag(Ap[:,j])
    Q[:,j] = Ap[:,j]/R[j,j]
  return Q, R

def gram_schmidt(A, b):
  Q, R = qrdec(A)
  Q_trans_b = np.transpose(Q)@b
  xs = backsub(R, Q_trans_b)
  print('Matrix A:')
  print(A)
  print('\nColumn Matrix b:')
  print(b)
  print('\nQR = A\nQ =', Q, '\nR =', R)
  print('\nA = QR = ', Q@R)
  print('\nSolution:')
  print(xs)
  print('\nSolution using np.linalg.solve():')
  print(np.linalg.solve(A, b))

def testcreate(n,val):
  A = np.arange(val,val+n*n).reshape(n,n)
  A = np.sqrt(A)
  bs = (A[0,:])**2.1
  return A, bs

A, bs = testcreate(4, 21)
gram_schmidt(A, bs)

Matrix A:
[[4.58257569 4.69041576 4.79583152 4.89897949]
 [5.         5.09901951 5.19615242 5.29150262]
 [5.38516481 5.47722558 5.56776436 5.65685425]
 [5.74456265 5.83095189 5.91607978 6.        ]]

Column Matrix b:
[24.45289367 25.67697243 26.90383729 28.13337297]

QR = A
Q = [[ 0.44095855  0.74266424 -0.48002199  0.15100169]
 [ 0.48112522  0.22782622  0.60503318 -0.58882127]
 [ 0.51818773 -0.21035281  0.38538773  0.73602108]
 [ 0.5527708  -0.5935459  -0.50496538 -0.29792779]] 
R = [[1.03923048e+01 1.05829568e+01 1.07701462e+01 1.09540595e+01]
 [0.00000000e+00 3.20069752e-02 6.28525184e-02 9.26292875e-02]
 [0.00000000e+00 0.00000000e+00 7.26503393e-05 2.06738475e-04]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 3.08319779e-07]]

A = QR =  [[4.58257569 4.69041576 4.79583152 4.89897949]
 [5.         5.09901951 5.19615242 5.29150262]
 [5.38516481 5.47722558 5.56776436 5.65685425]
 [5.74456265 5.83095189 5.91607978 6.        ]]

Solution:
[ 17690.96197192 -56886.91578401  60741.7649052

## Problem 4.31

Finding the eigenvectors of an upper triangular matrix $U$ is (much) easier than finding the eigenvectors of a general matrix. Most obviously, the first column of the identity matrix, $e_0$ as per Eq. (4.100), is an eigenvector of our triangular matrix. To see how
to find the rest, first partition our $n × n$ matrix as follows:

$$
U = \begin{pmatrix}
U_{00} & U_{0i} & R\\
0 & U_{ii} & S\\
0 & 0 & T
\end{pmatrix}
$$

where, crucially, $U_{ii}$ is a matrix element and $U_{00}$ is a submatrix; this means that $U_{0i}$ is a vector ($R, S,$ and $T$ won’t matter in what follows).

First, write down the eigenvector of $U$ with eigenvalue $U_{ii}$ as $\begin{pmatrix}
x_0 & 1 & 0
\end{pmatrix}^T$ and show that you can find $x_0$ by solving the upper-triangular system of equations:
$$
(U_{00} - U_{ii}I)x_0 = −U_{0i}
$$

Second, implement this approach programmatically to find all the eigenvectors of an upper-triangular matrix. (Return unit-norm versions of the eigenvectors.)

**Solution:** Consider the $n\times n$ upper triangular matrix

$$
U = \begin{pmatrix}
U_{00} & U_{01} & \dots & U_{0n}\\
0 & U_{11} & \dots & U_{1n}\\
\vdots & \vdots & \vdots & \vdots\\
0 & 0 & \dots & U_{nn}
\end{pmatrix}
$$

Let the eigenvector be $\begin{pmatrix}
x_0 & 1 & 0
\end{pmatrix}^T$

$$
\implies \begin{pmatrix}
U_{00} & U_{0i} & R\\
0 & U_{ii} & S\\
0 & 0 & T
\end{pmatrix}\begin{pmatrix}
x_0 \\ 1 \\ 0
\end{pmatrix} = U_{ii}\begin{pmatrix}
x_0 \\ 1 \\ 0
\end{pmatrix}\\
\implies U_{00}x_0 +U_{0i}=U_{ii}x_0\\
\implies (U_{ii}-U_{00})x_0 = U_{0i}\\
\text{or, } \boxed{(U_{00}-U_{ii})x_0 = -U_{0i}}
$$

In [13]:
import numpy as np

def find_eigenvectors(A):
  eigenvectors = []
  n = A.shape[0]

  for i in range(n-1, -1, -1):
    x = np.zeros(n)
    x[i] = 1
    for j in range(i - 1, -1, -1):
      x[j] = -(A[j, i] * x[i]) / A[j, j]
    x /= np.linalg.norm(x)
    eigenvectors.append(x)

  return eigenvectors

A = np.array([[1, 2, 3],
              [0, 4, 5],
              [0, 0, 6]])

eigenvectors = find_eigenvectors(A)

print('Matrix A:\n', A)
print('\nEigenvectors of A:')
for vect in eigenvectors:
  print(vect)

Matrix A:
 [[1 2 3]
 [0 4 5]
 [0 0 6]]

Eigenvectors of A:
[-0.88225755 -0.36760731  0.29408585]
[-0.89442719  0.4472136   0.        ]
[1. 0. 0.]
